In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt

from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard.writer import SummaryWriter

from sklearn.model_selection import train_test_split

from common import data
from common.training import training_loop, get_predictions

from algos import stoc
from algos import cbl

from common.models import simple_conv
from common import metrics
from common.models.modutils import load_model


c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
log_dir = "./outs/stoc"

horizon = 1024
stride = 512

batch_size = 8

num_epochs = list(range(1, 5))
out_feats = 64
k = 5

learning_rate = 1e-3

weight_decay = 1e-1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [5]:
paths = data.get_dataset_paths("../data")
train, test = train_test_split(paths, test_size=0.1, random_state=42)


In [6]:
m_data_train = data.Marconi100Dataset(train, scaling=data.Scaling.STANDARD)
m_data_test = data.Marconi100Dataset(test, scaling=data.Scaling.STANDARD)


Loading: 100%|██████████| 25/25 [00:14<00:00,  1.74it/s]


In [7]:
train_dataset = data.UnfoldedDataset(m_data_train, horizon=horizon, stride=stride)

test_dataset = data.UnfoldedDataset(m_data_test, horizon=horizon, stride=stride)


In [8]:
base_width = 128
down_layers = 2
dilations = [False, True]

backbone = simple_conv.Encoder(data.NUM_FEATURES, down_layers, base_width, dilations)
model = simple_conv.SimpleSegConv(backbone, out_feats)

optim = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
lr_sched = CosineAnnealingLR(optim, sum(num_epochs))

aug = cbl.pipeline(
    cbl.random_apply(cbl.left_to_right_flipping(1), 0.5),
    cbl.random_apply(cbl.crop_and_resize(1.5, 3), 0.8),
)

engine = cbl.CBLEngine(
    model=model,
    optimizer=optim,
    temperature=0.5,
    aug_1=aug,
    aug_2=aug,
    lr_scheduler=lr_sched,
    device=device,
)

stoc_engine = stoc.STOC(engine)


In [10]:
# with SummaryWriter(log_dir + "/supervised/logs") as writer:
#     stoc_engine.fit(
#         train_dataset,
#         test_dataset,
#         batch_size,
#         k,
#         num_epochs,
#         writer=writer,
#         save_path=log_dir + "/models",
#     )


stoc_engine.fit(train_dataset, test_dataset, batch_size, k, num_epochs)

In [ ]:
cmodel = metrics.default_cmodel()


In [ ]:
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)
scores, labels = get_predictions(stoc_engine.predict, test_loader)


In [ ]:
cmodel.fit(scores, labels).optimize()


In [ ]:
metrics.plot_cost(cmodel)


In [ ]:
metrics.plot_errors_curve(
    cmodel.false_positives, cmodel.false_negatives, cmodel.thresholds, figsize=(15, 5)
)


In [ ]:
pr, rec, f1 = metrics.precision_recall_f1(
    cmodel.false_positives, cmodel.false_negatives
)
metrics.plot_precision_recall_f1_curve(pr, rec, f1, cmodel.thresholds)
